# MAST30034 Applied Data Science Project 1

## Part 3: Modelling

### Import Libraries and Create Spark Session

In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import FloatType
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import FloatType
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain

In [ ]:
spark = (
    SparkSession.builder.appName("MAST30034 Project 1-3")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.executor.memory", "10g")
    .config("spark.driver.memory", "10g")
    .config("spark.sql.session.timeZone",  "Etc/UTC")
    .getOrCreate()
)

### Read In Data

In [ ]:
model_data = spark \
            .read.parquet('../data/curated/modelling/model_data')
model_test_data = spark \
            .read.parquet('../data/curated/modelling/model_test_data')

### Assemble and One-Hot Encode Features for Models

In [ ]:
# Features to be one-hot encoded
CATEGORICAL_LIST = ["month", "day", "day_of_week", "hour",
                    'PULocationID', "hvfhs_license_num"]

stage_list = []
encoded_cols = []

# Maps features into indices
for feature in CATEGORICAL_LIST:
    stage_list.append(StringIndexer(inputCol=feature,
                                    outputCol=f"{feature}_index"))

# Perform one-hot encoding
for feature in CATEGORICAL_LIST:
    stage_list.append(OneHotEncoder(inputCol=f"{feature}_index",
                                    outputCol=f"{feature}_vec"))
    encoded_cols.append(f"{feature}_vec")

# Create pipeline and pass all stages
encoding_pipeline = Pipeline(stages=stage_list)
sdf_transformed = encoding_pipeline.fit(model_data).transform(model_data)
sdf_test_transformed = encoding_pipeline \
                        .fit(model_data).transform(model_test_data)
(train_data, test_data) = sdf_transformed, sdf_test_transformed

feature_list = encoded_cols + ["rent"]
assembler = VectorAssembler(inputCols=feature_list, outputCol="features")

### Train Model and Generate Predictions 

Random Forest

In [ ]:
rf = RandomForestRegressor(
    labelCol="total_pickups",
    featuresCol="features",
    seed=0,
    numTrees=20)
pipeline_rf = Pipeline(stages=[assembler, rf])

# Train model
model_rf = pipeline_rf.fit(train_data)

# Make predictions
predictions_rf = model_rf.transform(test_data)

# Select example rows to display
predictions_rf.select("prediction", "total_pickups", "features").show(5)

Ordinary Least Squares (OLS)

In [ ]:
lr = LinearRegression(labelCol="total_pickups", featuresCol="features")
pipeline_lm = Pipeline(stages=[assembler, lr])

# Train model
model_lm = pipeline_lm.fit(train_data)

# Make predictions
predictions_lm = model_lm.transform(test_data)

# Select example rows to display
predictions_lm.select("prediction", "total_pickups", "features").show(5)

### Evaluate Model Performance

Code for OLS Output from: https://stackoverflow.com/questions/42935914/how-to-map-features-from-the-output-of-a-vectorassembler-back-to-the-column-name

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="total_pickups", predictionCol="prediction", metricName="rmse")

# Summary for OLS
rmse_lm = evaluator.evaluate(predictions_lm)
print("RMSE (OLS) on test data = %g" % rmse_lm)
lm = model_lm.stages[1]  # get lm model from pipeline
print(lm)
print(f"Adjusted R^2 for OLS: {lm.summary.r2adj}")

# Get name of features
attrs = sorted(
    (attr["idx"], attr["name"]) for attr in (chain(*predictions_lm
                                                   .schema[lm.summary
                                                             .featuresCol]
                                                   .metadata["ml_attr"]
                                                   ["attrs"].values())))

# Get OLS Output
lm_output = pd.DataFrame(
    data=[(name, lm.coefficients[idx],
          lm.summary.coefficientStandardErrors[idx],
          lm.summary.tValues[idx], lm.summary.pValues[idx])
          for idx, name in attrs],
    columns=["Feature", "Coefficient", "Standard Error", "t-value", "p-value"]
)
print("Last 5 Coefficients for Features: ")
print(lm_output.tail())

# Summary for Random Forest
rmse_rf = evaluator.evaluate(predictions_rf)
print("RMSE (Random Forest) on test data = %g" % rmse_rf)
rf = model_rf.stages[1]  # get rf model from pipeline
print(model_rf.stages[1])

# Get Random Forest Feature Importances
rf_output = pd.DataFrame(
    data=[(name, rf.featureImportances[idx]) for idx, name in attrs],
    columns=["Feature", "Feature Importance"]
)
rf_output.sort_values(by="Feature Importance", ascending=False).head()

### Error Analysis (Uber's Hourly Pickup, 15th April 2022, Upper West Side North)

In [ ]:
# Columns of predictiond data we want to extract
EXTRACT_COLS = ["month", "day", "hour", "PULocationID", "hvfhs_license_num",
                "total_pickups", "prediction"]

# Extract predictions from OLS
pred_sample_lm = predictions_lm.where(
    (F.col("month") == 4)
    & (F.col("day") == 15)
    & (F.col("PULocationID") == 238)  # Upper West Side North Zone Number
    & (F.col("hvfhs_license_num") == "HV0003")  # Uber's License Number
).select(*EXTRACT_COLS).toPandas()

# Extract predictions from Random Forest
pred_sample_rf = predictions_rf.where(
    (F.col("month") == 4)
    & (F.col("day") == 15)
    & (F.col("PULocationID") == 238)  # Upper West Side North Zone Number
    & (F.col("hvfhs_license_num") == "HV0003")  # Uber's License Number
).select(*EXTRACT_COLS).toPandas()

Plot Predictions vs Ground Truth (Pickups)

In [ ]:
plt.figure(figsize=(18, 5))
plt.rcParams.update({'font.size': 18})

N = 24
ind = np.arange(N)
width = 0.25

lm_vals = pred_sample_lm["prediction"]
bar1 = plt.bar(ind, lm_vals, width, color='r')

rf_vals = pred_sample_rf["prediction"]
bar2 = plt.bar(ind+width, rf_vals, width, color='b')

true_vals = pred_sample_lm["total_pickups"]
bar3 = plt.bar(ind+width*2, true_vals, width, color='g')

plt.xlabel("Hour")
plt.ylabel('Predicted Pickups')
plt.title("True vs Predicted Demand")

plt.xticks(ind+width, [f"{i:02}" for i in range(0, 24)])
plt.legend((bar1, bar2, bar3),
           ('Prediction (OLS)', 'Prediction (Random Forest)', 'True Pickups'))
plt.savefig("../plots/error_analysis.png")

### Full Distribution of Predictions

In [ ]:
# Extract full prediction columns
rf_dist = predictions_rf.select("prediction").toPandas()
lm_dist = predictions_lm.select("prediction").toPandas()

Density of Prediction (Random Forest)

In [ ]:
plt.rcParams.update({'font.size': 11})
rf_dist.plot.density(legend=None)
plt.title("Predictions by Random Forest")
plt.savefig("../plots/rf.png")

Density of Prediction (OLS)

In [ ]:
plt.rcParams.update({'font.size': 11})
lm_dist.plot.density(legend=None)
plt.title("Predictions by OLS")
plt.savefig("../plots/ols.png")